# Modelagem de Tópicos — LDA (scikit-learn)

> Prof. Matheus C. Pestana (FGV Comunicação Rio)

#### Caminho básico

1) Pré-processamento leve

2) Vetorização (Bag-of-Words)

3) LDA (Latent Dirichlet Allocation) em scikit-learn

4) BERTopic (com embeddings) para comparação - outro notebook

##### Observações didáticas:
- LDA trabalha sobre a matriz termo-documento (contagens), buscando "misturas" de tópicos.
- BERTopic combina embeddings + clustering + c-TF-IDF para temas mais coerentes em muitos casos.
- Rodar BERTopic pode levar mais tempo na 1ª vez (download do modelo de embeddings).


In [ ]:
import random
import re
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Vamos carregar uma base chamada 20newsgroups, de listas de emails temáticas. É possível usar qualquer outra base do pandas para fazer a alteração.

In [ ]:
print("[INFO] Carregando 20 Newsgroups (apenas 'train')...")
raw = fetch_20newsgroups(subset="train", remove=("headers", "footers", "quotes"))
docs = raw.data
print(f"[INFO] Total de documentos disponíveis: {len(docs)}")

# Vamos limitar a 2000 para fins de exemplificação
max_n = 2000
if len(docs) > max_n:
    random.seed(42)
    idx = random.sample(range(len(docs)), max_n)
    docs = [docs[i] for i in idx]
print(f"[INFO] Usando {len(docs)} documentos para a aula.")


## 1) Limpeza do texto (obrigatória para LDA)

Vamos limpar o texto, removendo múltiplos espaços em branco, mantendo letras e pontuação básica. Poderíamos remover stopwords também, mas isso acarretaria em "overclean". O próprio vetorizador cuidará disso...

In [ ]:
def simple_clean(s):
    s = s or "" # Transforma em texto o que não for
    s = re.sub(r"\s+", " ", s)  # espaços múltiplos em branco -> 1
    s = s.strip() # Remove os espaços do início e do final
    return s

docs = [simple_clean(d) for d in docs]
print("[INFO] Limpeza leve concluída.")

# Qual a outra forma de limpar os documentos?

## 2) Vetorização

A vetorização é o processo de transformar textos — que são dados não estruturados — em uma representação numérica que os algoritmos conseguem entender. Em vez de lidar com frases ou parágrafos diretamente, o computador cria uma matriz de termos por documentos: cada coluna representa uma palavra (ou combinação de palavras) e cada linha representa um documento. Assim, a presença, a frequência ou o peso desses termos são registrados em números que podem ser usados em modelos como LDA ou BERTopic.

Nesse processo, usamos algumas técnicas importantes:

•	Stopwords: são palavras muito frequentes, mas que carregam pouca informação semântica (como “the”, “of”, “em”, “de”). Retirá-las ajuda a reduzir ruído e destacar termos realmente relevantes.

•	N-gramas: enquanto unigramas consideram apenas palavras isoladas, bigrams e trigrams capturam combinações de 2 ou 3 palavras consecutivas (ex.: “inteligência artificial”, “ciência de dados”), o que enriquece a interpretação dos tópicos.

•	min_df: define a frequência mínima para que um termo seja mantido. Palavras que aparecem em pouquíssimos documentos são descartadas por não contribuírem de forma consistente.

•	max_df: faz o oposto, eliminando palavras que aparecem em quase todos os documentos, pois tendem a não diferenciar os textos.

Em resumo, vetorização é o passo em que transformamos linguagem em números, equilibrando o que manter e o que descartar para que os modelos consigam identificar padrões significativos.

In [ ]:
# Como seria o mesmo conjunto de textos (mais simples) vetorizado em ambas?
textos = [
          'oi, meu nome é joão',
          'oi, tudo bem?',
          '"joão e maria" é uma história para crianças',
          'nossa história não acabará'
          ]

count = CountVectorizer()
X_count = count.fit_transform(textos)
print(pd.DataFrame(X_count.toarray(), columns=count.get_feature_names_out()))

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(textos)
print(pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out()))


### CountVectorizer

O CountVectorizer é a forma mais simples de vetorização de textos: ele constrói uma matriz em que cada coluna corresponde a um termo (palavra ou n-grama) e cada célula contém a contagem absoluta de vezes que aquele termo aparece em um documento. Por exemplo, se a palavra “dados” aparece 3 vezes em um texto, a célula correspondente recebe o valor 3. Isso gera uma matriz esparsa (muitos zeros), que funciona bem para modelos como LDA, pois esse método parte da ideia de que os documentos são misturas de tópicos e tópicos são misturas de palavras frequentes.

### TfidfVectorizer

O TfidfVectorizer (Term Frequency–Inverse Document Frequency) também cria uma matriz termo-documento, mas em vez de apenas contar frequências, ele pesa cada termo de acordo com sua importância relativa.

•	O TF (Term Frequency) mede quantas vezes a palavra aparece em um documento.
•	O IDF (Inverse Document Frequency) mede o quão rara a palavra é no conjunto de documentos: termos muito comuns (como “dia”, “ano”, “gente”) recebem peso baixo, enquanto termos mais raros e distintivos recebem peso alto.

Assim, o TF-IDF dá um peso maior a termos que ajudam a diferenciar documentos, e não apenas a termos muito repetidos. Essa abordagem costuma ser mais eficaz em tarefas como classificação, busca de informação e clustering com embeddings, pois favorece termos discriminativos em vez de palavras genéricas.

#### Comparação resumida

•	CountVectorizer: simples, usa contagem bruta de palavras

•	TfidfVectorizer: usa ponderação TF-IDF (frequência + raridade)

In [ ]:
vectorizer = CountVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    max_df=0.95,
    min_df=5
)
X_count = vectorizer.fit_transform(docs)
terms_count = np.array(vectorizer.get_feature_names_out())
print(f"[INFO] Matriz termo-documento: {X_count.shape} (docs x termos)")

vectorizer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    max_df=0.95,
    min_df=5
)
X_tfidf = vectorizer.fit_transform(docs)
terms_tfdf = np.array(vectorizer.get_feature_names_out())
print(f"[INFO] Matriz termo-documento: {X_tfidf.shape} (docs x termos)")

In [ ]:
print(f"Com CountVectorizer:")
print(pd.DataFrame(X_count.toarray(), columns=terms_count).head())
print(f"Com TfidfVectorizer:")
print(pd.DataFrame(X_tfidf.toarray(), columns=terms_tfdf).head())

## 3) LDA - Alocação Latente de Dirichlet



In [ ]:
n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics,
    learning_method="batch",
    random_state=42
)
print("[INFO] Ajustando LDA...")
W = lda.fit_transform(X_tfidf)  # distribuição doc->tópicos
H = lda.components_             # distribuição tópico->termos (contagens "suavizadas")

In [ ]:
def show_top_words(H, terms, topn=12):
    for k, row in enumerate(H):
        top_idx = row.argsort()[::-1][:topn]
        top_terms = terms[top_idx]
        print(f"\n[Tópico {k}]")
        print(", ".join(top_terms))

print("\n======== LDA: Top palavras por tópico ========")
show_top_words(H, terms_tfdf, topn=12)

### Como achar o melhor número de tópicos?

#### ELBO (Evidence Lower Bound)

O ELBO, ou Evidence Lower Bound, é uma medida interna usada quando aplicamos inferência variacional para ajustar modelos probabilísticos, como o LDA.

•	Em teoria, gostaríamos de calcular a verossimilhança (likelihood) completa dos dados dado o modelo, mas isso é inviável porque exige integrais muito complexas.

•	Em vez disso, usamos uma aproximação variacional: escolhemos uma distribuição aproximada para os parâmetros e otimizamos essa aproximação.

•	O ELBO é justamente o limite inferior da verossimilhança — quanto maior o ELBO, melhor a aproximação variacional está representando a probabilidade verdadeira dos dados.

Resumindo... o ELBO serve como critério de otimização e diagnóstico interno. Modelos com maior ELBO estão explicando melhor os dados dentro do espaço de aproximação escolhido.

#### Perplexidade

A perplexidade é uma métrica mais interpretável, derivada da log-verossimilhança, que mede o quão “surpreso” o modelo fica ao ver um novo conjunto de dados.

•	Formalmente, é a exponencial da entropia média dos documentos no modelo.

•	Intuitivamente, valores baixos de perplexidade indicam que o modelo prevê bem as distribuições de palavras nos documentos, ou seja, que ele está menos “perplexo” com o que aparece.

Resumindo... quanto menor a perplexidade, melhor o modelo está capturando a estrutura dos dados.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

# Dividir em treino/teste para avaliar generalização
X_train, X_test = train_test_split(X_tfidf, test_size=0.2, random_state=42)

k_candidates = [4, 5, 6, 7, 8, 10, 12, 15, 20]  # ajuste livremente
results = []

print("\n[INFO] Avaliando diferentes k para LDA...")
for k in k_candidates:
    lda_k = LatentDirichletAllocation(
        n_components=k,
        learning_method="batch",
        random_state=42
    )
    lda_k.fit(X_train)

    train_ll = lda_k.score(X_train)         # log-likelihood (ELBO), maior melhor
    test_perp = lda_k.perplexity(X_test)    # perplexidade no teste, menor melhor

    results.append({"k": k, "train_loglik": train_ll, "test_perplexity": test_perp})
    print(f"  k={k:>2} | train_loglik={train_ll:>12.2f} | test_perplexity={test_perp:>10.2f}")

# Organizar resultados
res_df = pd.DataFrame(results).sort_values(["test_perplexity", "train_loglik"], ascending=[True, False])
print("\n[INFO] Resultados ordenados (melhor perplexidade primeiro):")
print(res_df)

# Sugerir k (menor perplexidade)
best_k = res_df.iloc[0]["k"]
print(f"\n[SUGESTÃO] Melhor k pela menor perplexidade de teste: k={int(best_k)}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot([r["k"] for r in results], [r["test_perplexity"] for r in results], marker="o")
plt.xlabel("k (número de tópicos)")
plt.ylabel("Perplexidade (teste) — menor é melhor")
plt.title("Seleção de k para LDA (perplexidade)")
plt.show()

# Qual o problema dessas métricas?

•	O ELBO é usado durante o treinamento (critério de ajuste interno).

•	A perplexidade é usada como métrica de avaliação (quanto o modelo generaliza bem para novos documentos).

•	Contudo, ambas não garantem interpretabilidade: às vezes um modelo com perplexidade melhor pode gerar tópicos pouco coerentes. Por isso, sempre se complementa com análise qualitativa ou métricas de coerência de tópicos.

# BERTopic

https://colab.research.google.com/drive/1JyYNCQel1YaePZ_ARNz1CQT55ijYi5YQ#scrollTo=qOgead0wzCRR